In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import csv

In [2]:
CHROMEDRIVER_PATH = "/Users/parakhchaudhary/chromedriver_135/chromedriver-mac-x64/chromedriver"

options = Options()
options.add_argument("--headless")
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

base_url = "https://www.shl.com/products/product-catalog/?start={}&type=1"
data = []

In [3]:

for offset in range(0, 372, 12):
    print(f"Scraping offset {offset}")
    driver.get(base_url.format(offset))

    time.sleep(2)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tables = soup.find_all("table")

    if len(tables) < 2:
        continue

    table = tables[1]  # Individual Test Solutions table
    rows = table.find_all("tr")[1:]

    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 4:
            continue

        test_name_tag = cols[0].find("a")
        test_name = test_name_tag.text.strip()
        test_url = test_name_tag['href']
        remote_testing = "Yes" if cols[1].find("svg") else "No"
        adaptive = "Yes" if cols[2].find("svg") else "No"
        test_types = ''.join(span.text for span in cols[3].find_all("span"))

        data.append({
            "name": test_name,
            "url": test_url,
            "remote_testing": remote_testing,
            "adaptive_support": adaptive,
            "test_type": test_types
        })

driver.quit()

Scraping offset 0
Scraping offset 12
Scraping offset 24
Scraping offset 36
Scraping offset 48
Scraping offset 60
Scraping offset 72
Scraping offset 84
Scraping offset 96
Scraping offset 108
Scraping offset 120
Scraping offset 132
Scraping offset 144
Scraping offset 156
Scraping offset 168
Scraping offset 180
Scraping offset 192
Scraping offset 204
Scraping offset 216
Scraping offset 228
Scraping offset 240
Scraping offset 252
Scraping offset 264
Scraping offset 276
Scraping offset 288
Scraping offset 300
Scraping offset 312
Scraping offset 324
Scraping offset 336
Scraping offset 348
Scraping offset 360


In [4]:

with open("shl_individual_test_solutions_New.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print("✅ Done. Saved to shl_individual_test_solutions.csv")


✅ Done. Saved to shl_individual_test_solutions.csv
